In [6]:
n = 4
board = [["."] * n for i in range(n)]
joinedBoard = ["".join(boardRow) for boardRow in board]
print(board)
print(joinedBoard)

test = ["."] * n
print(test)

[['.', '.', '.', '.'], ['.', '.', '.', '.'], ['.', '.', '.', '.'], ['.', '.', '.', '.']]
['....', '....', '....', '....']
['.', '.', '.', '.']


In [4]:
board = ["." * n for i in range(n)]
print(board)

board = ["." * i for i in range(n)]
print(board)

['....', '....', '....', '....']
['', '.', '..', '...']


In [7]:
print(ord('a') - ord('a'))
print(ord('a'))
print(ord('b'))
print(ord('z'))

0
97
98
122


In [1]:
arr = []
arr.append(1)
arr.append(2)
arr.append(3)
print(arr)

AttributeError: 'list' object has no attribute 'push'